In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Adidas Sales Analysis').getOrCreate()

In [4]:
df = spark.read.csv('/content/Adidas_Clean_Dataset.csv', header = True, inferSchema=True)
df.show()

+---+-----------+-----------+------------+---------+--------+--------+--------------------+--------------+----------+-----------+----------------+----------------+------------+
|_c0|   Retailer|Retailer_ID|Invoice_Date|   Region|   State|    City|             Product|Price_per_Unit|Units_Sold|Total_Sales|Operating_Profit|Operating_Margin|Sales_Method|
+---+-----------+-----------+------------+---------+--------+--------+--------------------+--------------+----------+-----------+----------------+----------------+------------+
|  0|Foot Locker|    1185732|  01-01-2020|Northeast|New York|New York|Men's Street Foot...|          50.0|     1,200|   600000.0|        300000.0|             50%|    In-store|
|  1|Foot Locker|    1185732|  02-01-2020|Northeast|New York|New York|Men's Athletic Fo...|          50.0|     1,000|   500000.0|        150000.0|             30%|    In-store|
|  2|Foot Locker|    1185732|  03-01-2020|Northeast|New York|New York|Women's Street Fo...|          40.0|     1,00

In [5]:
df = df.toDF(*[col.lower() for col in df.columns])
df.show()

+---+-----------+-----------+------------+---------+--------+--------+--------------------+--------------+----------+-----------+----------------+----------------+------------+
|_c0|   retailer|retailer_id|invoice_date|   region|   state|    city|             product|price_per_unit|units_sold|total_sales|operating_profit|operating_margin|sales_method|
+---+-----------+-----------+------------+---------+--------+--------+--------------------+--------------+----------+-----------+----------------+----------------+------------+
|  0|Foot Locker|    1185732|  01-01-2020|Northeast|New York|New York|Men's Street Foot...|          50.0|     1,200|   600000.0|        300000.0|             50%|    In-store|
|  1|Foot Locker|    1185732|  02-01-2020|Northeast|New York|New York|Men's Athletic Fo...|          50.0|     1,000|   500000.0|        150000.0|             30%|    In-store|
|  2|Foot Locker|    1185732|  03-01-2020|Northeast|New York|New York|Women's Street Fo...|          40.0|     1,00

In [6]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- retailer: string (nullable = true)
 |-- retailer_id: integer (nullable = true)
 |-- invoice_date: string (nullable = true)
 |-- region: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- product: string (nullable = true)
 |-- price_per_unit: double (nullable = true)
 |-- units_sold: string (nullable = true)
 |-- total_sales: double (nullable = true)
 |-- operating_profit: double (nullable = true)
 |-- operating_margin: string (nullable = true)
 |-- sales_method: string (nullable = true)



In [13]:
from pyspark.sql.functions import *
df = df.withColumn("invoice_date", to_date("invoice_date", "dd-MM-yyyy"))
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- retailer: string (nullable = true)
 |-- retailer_id: integer (nullable = true)
 |-- invoice_date: date (nullable = true)
 |-- region: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- product: string (nullable = true)
 |-- price_per_unit: double (nullable = true)
 |-- units_sold: string (nullable = true)
 |-- total_sales: double (nullable = true)
 |-- operating_profit: double (nullable = true)
 |-- operating_margin: string (nullable = true)
 |-- sales_method: string (nullable = true)



In [14]:
df.createOrReplaceTempView('adidas')

In [15]:
result = spark.sql("""
  select * from adidas
""")
result.show()

+---+-----------+-----------+------------+---------+--------+--------+--------------------+--------------+----------+-----------+----------------+----------------+------------+
|_c0|   retailer|retailer_id|invoice_date|   region|   state|    city|             product|price_per_unit|units_sold|total_sales|operating_profit|operating_margin|sales_method|
+---+-----------+-----------+------------+---------+--------+--------+--------------------+--------------+----------+-----------+----------------+----------------+------------+
|  0|Foot Locker|    1185732|  2020-01-01|Northeast|New York|New York|Men's Street Foot...|          50.0|     1,200|   600000.0|        300000.0|             50%|    In-store|
|  1|Foot Locker|    1185732|  2020-01-02|Northeast|New York|New York|Men's Athletic Fo...|          50.0|     1,000|   500000.0|        150000.0|             30%|    In-store|
|  2|Foot Locker|    1185732|  2020-01-03|Northeast|New York|New York|Women's Street Fo...|          40.0|     1,00

Maximum Number of units sold

In [27]:
result1 = spark.sql("""
  select *
  from adidas
  order by units_sold desc
  limit 1;
""")
result1.show()

+----+-----------+-----------+------------+---------+------------+------------+--------------------+--------------+----------+-----------+----------------+----------------+------------+
| _c0|   retailer|retailer_id|invoice_date|   region|       state|        city|             product|price_per_unit|units_sold|total_sales|operating_profit|operating_margin|sales_method|
+----+-----------+-----------+------------+---------+------------+------------+--------------------+--------------+----------+-----------+----------------+----------------+------------+
|8169|Foot Locker|    1185732|  2021-11-28|Northeast|Pennsylvania|Philadelphia|Men's Athletic Fo...|          45.0|        99|     4455.0|          1737.0|             39%|      Outlet|
+----+-----------+-----------+------------+---------+------------+------------+--------------------+--------------+----------+-----------+----------------+----------------+------------+



Count of Products with Product Name

In [10]:
result2 = spark.sql("""
  select distinct product as products, count(product) as total_products
  from adidas
  group by Product;
""")
result2.show()

+--------------------+--------------+
|            products|total_products|
+--------------------+--------------+
|Men's Athletic Fo...|          1610|
|Women's Athletic ...|          1606|
|       Men's Apparel|          1606|
|     Women's Apparel|          1608|
|Women's Street Fo...|          1608|
|Men's Street Foot...|          1610|
+--------------------+--------------+



Count of City with City Name where products sold

In [19]:
result3 = spark.sql("""
  select distinct city, count(city) city_count
  from adidas
  group by city;
""")
result3.show()

+-------------+----------+
|         city|city_count|
+-------------+----------+
|   Charleston|       288|
|      Phoenix|       216|
|        Omaha|       144|
|    Anchorage|       144|
|       Dallas|       216|
|   Manchester|       216|
| Philadelphia|       216|
|   Louisville|       144|
|  Los Angeles|       216|
| Indianapolis|       144|
|San Francisco|       216|
|Oklahoma City|       216|
|      Detroit|       144|
|     Portland|       360|
|       Albany|       144|
|        Boise|       216|
|     Cheyenne|       144|
|    St. Louis|       144|
|   Birmingham|       216|
|   Burlington|       216|
+-------------+----------+
only showing top 20 rows



Maximum Profit by Adidas

In [33]:
result4 = spark.sql("""
  select product, max(operating_profit) as max_profit
  from adidas
  group by 1;
""")
result4.show()

+--------------------+----------+
|             product|max_profit|
+--------------------+----------+
|Men's Athletic Fo...|  285000.0|
|Women's Athletic ...|  216000.0|
|       Men's Apparel|  281250.0|
|     Women's Apparel|  371250.0|
|Women's Street Fo...|  266000.0|
|Men's Street Foot...|  390000.0|
+--------------------+----------+



City with Maximum Unit Sold by Adidas

In [18]:
result5 = spark.sql("""
  select distinct city, max(Units_sold) as Max_Units_Sold
  from adidas
  group by city;
""")
result5.show()

+-----------+--------------+
|       city|Max_Units_Sold|
+-----------+--------------+
|     Albany|           950|
|Albuquerque|            99|
|  Anchorage|            98|
|    Atlanta|            98|
|  Baltimore|            91|
|   Billings|            98|
| Birmingham|            98|
|      Boise|           925|
|     Boston|            98|
| Burlington|            99|
| Charleston|            98|
|  Charlotte|           975|
|   Cheyenne|            91|
|    Chicago|            94|
|   Columbus|            98|
|     Dallas|           975|
|     Denver|            98|
| Des Moines|            98|
|    Detroit|            98|
|      Fargo|            98|
+-----------+--------------+
only showing top 20 rows



Information about Retailers

In [22]:
result6 = spark.sql("""
  select retailer, count(*) retailer_count
  from adidas
  group by 1;
""")
result6.show()

+-------------+--------------+
|     retailer|retailer_count|
+-------------+--------------+
|      Walmart|           626|
|  Foot Locker|          2637|
|    West Gear|          2374|
|       Kohl's|          1030|
|       Amazon|           949|
|Sports Direct|          2032|
+-------------+--------------+



Maximum Sales by Adidas Retailers

In [23]:
result7 = spark.sql("""
  select distinct retailer, max(total_sales) as total_sales
  from adidas
  group by Retailer;
""")
result7.show()

+-------------+-----------+
|     retailer|total_sales|
+-------------+-----------+
|      Walmart|   825000.0|
|  Foot Locker|   765000.0|
|    West Gear|   825000.0|
|       Kohl's|   520000.0|
|       Amazon|   618750.0|
|Sports Direct|   698750.0|
+-------------+-----------+



Maximum Price per Unit by Name of Product in Adidas

In [24]:
result8 = spark.sql("""
  select product, max(price_per_unit) as max_price_per_unit
  from adidas
  group by 1;
""")
result8.show()

+--------------------+------------------+
|             product|max_price_per_unit|
+--------------------+------------------+
|Men's Athletic Fo...|              95.0|
|Women's Athletic ...|              90.0|
|       Men's Apparel|              95.0|
|     Women's Apparel|             110.0|
|Women's Street Fo...|              95.0|
|Men's Street Foot...|              95.0|
+--------------------+------------------+



Information About the last Products sold by Adidas in Year 2021

In [26]:
result9 = spark.sql("""
  select *
  from adidas
  where Invoice_Date = "2021-12-31";
""")
result9.show()

+----+-----------+-----------+------------+---------+------------+------------+-------------+--------------+----------+-----------+----------------+----------------+------------+
| _c0|   retailer|retailer_id|invoice_date|   region|       state|        city|      product|price_per_unit|units_sold|total_sales|operating_profit|operating_margin|sales_method|
+----+-----------+-----------+------------+---------+------------+------------+-------------+--------------+----------+-----------+----------------+----------------+------------+
| 498|Foot Locker|    1185732|  2021-12-31|Northeast|Pennsylvania|Philadelphia|Men's Apparel|          70.0|       175|   122500.0|         42875.0|             35%|      Outlet|
|1200|     Amazon|    1185732|  2021-12-31|Northeast|       Maine|    Portland|Men's Apparel|          55.0|       125|    68750.0|         17188.0|             25%|      Outlet|
|4386|Foot Locker|    1185732|  2021-12-31|Northeast|Pennsylvania|Philadelphia|Men's Apparel|          63

Count on No of Products sold on last day of 2021

In [28]:
result10 = spark.sql("""
  select count(*) as No_oF_Products
  from adidas
  where Invoice_Date = "2021-12-31";
""")
result10.show()


+--------------+
|No_oF_Products|
+--------------+
|             5|
+--------------+



Total count of Sales method each City wise

In [29]:
result11 = spark.sql("""
  select distinct sales_method, city, count(sales_method) as total_methods
  from adidas
  group by Sales_Method, city
  order by sales_method;
""")
result11.show()

+------------+--------------+-------------+
|sales_method|          city|total_methods|
+------------+--------------+-------------+
|    In-store|        Boston|           72|
|    In-store|      Cheyenne|           72|
|    In-store|      Hartford|           72|
|    In-store|        Denver|           72|
|    In-store|       Orlando|           50|
|    In-store|    Burlington|           72|
|    In-store|   New Orleans|           20|
|    In-store|    Charleston|           72|
|    In-store|      Portland|           72|
|    In-store|       Detroit|           72|
|    In-store|     St. Louis|           72|
|    In-store|         Miami|           72|
|    In-store|   Minneapolis|           26|
|    In-store|Salt Lake City|           72|
|    In-store|      Richmond|           72|
|    In-store|       Seattle|           72|
|    In-store|        Albany|           72|
|    In-store|  Indianapolis|           72|
|    In-store|         Fargo|           59|
|    In-store|    Manchester|   

Maximum and Minimum Operting Margin for each City

In [32]:
result12 = spark.sql("""
  select city, max(Operating_margin), min(Operating_margin)
  from adidas
  group by city
  order by city;
""")
result12.show()

+-----------+---------------------+---------------------+
|       city|max(Operating_margin)|min(Operating_margin)|
+-----------+---------------------+---------------------+
|     Albany|                  65%|                  25%|
|Albuquerque|                  55%|                  25%|
|  Anchorage|                  54%|                  15%|
|    Atlanta|                  60%|                  25%|
|  Baltimore|                  55%|                  30%|
|   Billings|                  75%|                  25%|
| Birmingham|                  80%|                  30%|
|      Boise|                  69%|                  25%|
|     Boston|                  60%|                  25%|
| Burlington|                  60%|                  30%|
| Charleston|                  65%|                  25%|
|  Charlotte|                  65%|                  35%|
|   Cheyenne|                  65%|                  25%|
|    Chicago|                  65%|                  30%|
|   Columbus| 